<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# Импортируем нужные библиотеки
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
import seaborn as sns

import math
import re
import time

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from tqdm import notebook
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle

In [2]:
RANDOM_STATE = 12345

In [3]:
# Напишем функцию для первичной проверки данных
def check_data(data_df):
    print ('\033[1m' + 'Изучим исходные данные'+ '\033[0m')
    print(data_df.info())
    #print(data_df.shape)

    missed_cells = data_df.isnull().sum().sum()/(data_df.shape[0]*(data_df.shape[1]-1))
    missed_rows = sum(data_df.isnull().sum(axis = 1)>0)/data_df.shape[0]
    print ('\033[1m' + '\nПроверка пропусков'+ '\033[0m')
    print ('Количество пропусков: {:.0f}'.format(data_df.isnull().sum().sum()))
    print ('Доля пропусков: {:.1%}'.format(missed_cells)+ '\033[0m')
    print ('Доля строк содержащих пропуски: {:.1%}'.format(missed_rows))

    ## Проверим дубликаты
    print ('\033[1m' + '\nПроверка на дубликаты'+ '\033[0m')
    print('Количество полных дубликатов: ', data_df.duplicated().sum())

    ## Посмотрим на сами данные
    print ('\033[1m' + '\nПервые пять строк датасета'+ '\033[0m')
    display(data_df.head())

    print('\033[1m' + '\nОписание количественных данных:'+ '\033[0m')
    display(data_df.describe().T)

In [4]:
# Считываем данные из csv-файла в датафрейм и сохраняем в переменной data
try:
    data_raw = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data_raw = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [5]:
data_raw.head(3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


In [6]:
# описательный анализ
check_data(data_raw)

Изучим исходные данные
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None

Проверка пропусков
Количество пропусков: 0
Доля пропусков: 0.0%
Доля строк содержащих пропуски: 0.0%

Проверка на дубликаты
Количество полных дубликатов:  0

Первые пять строк датасета


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0



Описание количественных данных:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,159292.0,79725.697242,46028.837471,0.0,39872.75,79721.5,119573.25,159450.0
toxic,159292.0,0.101612,0.302139,0.0,0.00,0.0,0.00,1.0


In [7]:
data_raw = data_raw.drop(['Unnamed: 0'], axis=1)

In [8]:
data_raw.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


Проверим баланс классов

In [9]:
toxic_balance = data_raw.groupby(['toxic']).agg({'toxic':'count'})
toxic_balance.columns = ['count']
toxic_balance['percent'] =  round(toxic_balance['count'] / len(data_raw) * 100, 2)

In [10]:
toxic_balance

,count,percent
toxic,,
0,143106,89.84
1,16186,10.16


Видим, что токсичных комментариев намного меньше (всего лишь 10 процентов).

In [11]:
def clear_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip().lower()
    return text

In [12]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [13]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemm_list = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    lemm_text = " ".join(lemm_list)

    return lemm_text

In [14]:
data = data_raw.copy()

In [15]:
data['text_cleaned'] = data['text'].apply(clear_text)

In [16]:
corpus = data['text_cleaned'].values

In [17]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
corpus_lemmatized = np.vectorize(lemmatize)(corpus)

In [20]:
print("Исходный текст:", data['text'][0])
print("Очищенный текст:", data['text_cleaned'][0])
print("Очищенный и лемматизированный текст:", corpus_lemmatized[0])

Исходный текст: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Очищенный текст: explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now
Очищенный и лемматизированный текст: explanation why the edits make under my username hardcore metallica fan be revert they werent vandalism just closure on some gas after i vote at new york doll fac and please dont remove the template from the talk page since im retire now


In [21]:
nltk.download('stopwords')
stopwords = nltk_stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

In [22]:
features = corpus
target = data['toxic']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=RANDOM_STATE)

In [24]:
# Проверка
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((127433,), (31859,), (127433,), (31859,))

In [25]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(x_train)
tf_idf_test = count_tf_idf.transform(x_test)

In [26]:
# Проверка
tf_idf_train.shape, tf_idf_test.shape

((127433, 192962), (31859, 192962))

In [27]:
# СФОРМИРУЕМ ШАГИ ПАПЛАЙНА
pipe = Pipeline(
    [
        ("model", LogisticRegression(random_state = RANDOM_STATE))
    ]
    )

In [28]:
param_grid = [
    {
        'model': [LogisticRegression(random_state=RANDOM_STATE)],
        'model__penalty': ['l1', 'l2'],
        'model__C': list(range(1,15,3)),
        'model__solver' : ['liblinear'],
        'model__max_iter' : list(range(100,1000,100))
    },
    {
        'model': [LinearSVC(random_state = RANDOM_STATE, class_weight={0: 1, 1: 9})],
        'model__penalty': ['l1', 'l2'],
        'model__C': list(range(1,15,3)),
        'model__max_iter' : list(range(100,1000,100))

    }
]

In [29]:
grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', cv=3, n_jobs=-1)
best_grid = grid.fit(tf_idf_train, y_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/svm/_classes.py", line 234, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py", line 974, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py", line 830, in _get_liblinear_solver_typ

Best parameters is: {'model': LogisticRegression(C=4, penalty='l1', random_state=12345, solver='liblinear'), 'model__C': 4, 'model__max_iter': 100, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Best score is: 0.7707309163429882


In [30]:
results_pipe = pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score', ascending=True)

In [31]:
# РЕЗУЛЬТАТ ВСЕХ МОДЕЛЕЙ
results_pipe = pd.DataFrame(grid.cv_results_)[['mean_fit_time', 'mean_score_time', 'param_model', 'params', 'mean_test_score', 'rank_test_score']]

In [32]:
results_pipe.sort_values(by='rank_test_score', ascending=True)

,mean_fit_time,mean_score_time,param_model,params,mean_test_score,rank_test_score
28,1.433482,0.021149,"LogisticRegression(C=4, penalty='l1', random_s...","{'model': LogisticRegression(C=4, penalty='l1'...",0.770731,1
30,1.442491,0.015334,"LogisticRegression(C=4, penalty='l1', random_s...","{'model': LogisticRegression(C=4, penalty='l1'...",0.770731,1
24,1.555027,0.023112,"LogisticRegression(C=4, penalty='l1', random_s...","{'model': LogisticRegression(C=4, penalty='l1'...",0.770731,1
32,1.683511,0.021881,"LogisticRegression(C=4, penalty='l1', random_s...","{'model': LogisticRegression(C=4, penalty='l1'...",0.770731,1
22,1.395786,0.015885,"LogisticRegression(C=4, penalty='l1', random_s...","{'model': LogisticRegression(C=4, penalty='l1'...",0.770731,1
...,...,...,...,...,...,...
140,0.024512,0.000000,"LinearSVC(class_weight={0: 1, 1: 9}, random_st...","{'model': LinearSVC(class_weight={0: 1, 1: 9},...",NaN,176
142,0.023358,0.000000,"LinearSVC(class_weight={0: 1, 1: 9}, random_st...","{'model': LinearSVC(class_weight={0: 1, 1: 9},...",NaN,177
144,0.025546,0.000000,"LinearSVC(class_weight={0: 1, 1: 9}, random_st...","{'model': LinearSVC(class_weight={0: 1, 1: 9},...",NaN,178
148,0.022183,0.000000,"LinearSVC(class_weight={0: 1, 1: 9}, random_st...","{'model': LinearSVC(class_weight={0: 1, 1: 9},...",NaN,179


In [33]:
print(f'Лучшие параметры: ', grid.best_params_)
print(f'Метрика MRSE для лучших параметров дерева решений равно: %.4f' % grid.best_score_)

Лучшие параметры:  {'model': LogisticRegression(C=4, penalty='l1', random_state=12345, solver='liblinear'), 'model__C': 4, 'model__max_iter': 100, 'model__penalty': 'l1', 'model__solver': 'liblinear'}
Метрика MRSE для лучших параметров дерева решений равно: 0.7707


In [34]:
model = LogisticRegression(C=4, max_iter=100, penalty='l1', random_state=RANDOM_STATE, solver='liblinear')

In [35]:
model.fit(tf_idf_train, y_train)
predicted_test = model.predict(tf_idf_test)

In [36]:
f1_score(y_test, predicted_test)

0.7886082386845227

Для оценки модели на адекватность, модель должна быть умнее тривиальной модели. В нашем исследовании в качестве тривиальной модели будем использовать модель DummyClassifier со статегией constant, которая возвращает 1.

In [37]:
model_dummy = DummyClassifier(random_state=RANDOM_STATE, strategy = 'constant', constant=1)
model_dummy.fit(x_train, y_train)
predicted_dummy_test = model_dummy.predict(x_test)

In [38]:
f1_score(y_test.values, predicted_dummy_test)

0.1841032802302716

## Выводы

В рамках данного исследования были загружены и изучены данные о комментариях клиентов. Данные были загружены и проверены на корректность.

На тренировочной выборке были обучены модели линейной регресии, решающего дерева и подобраны гиперпараметры. По значению F1 меры на тренировочной выборке, была выбрана модель с наибольшим значением F1 меры. Значение F1 меры на тестовой выборке: 0.79, что выше поставленной задачи. Выбранная модель была проверена на корректность, сравеннием с Dummy моделью, которая будет предсказывать постоянное значение (1). Так как модель показала результат лучше чем Dummy модель, ее использование оправдано. В дальнейшем следует продолжить исследование, используя более сложные модели, например BERT.